# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
#DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.
DATA_FOLDER = "./Data"

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Strategy

We will import and wrangle the data for each country separately and then combine it into a single data frame.

As the data files already contain variables with nation wide statistics, we will use these values instead of manually aggregating the data city by city.

In [36]:
def ebola_csv_iter(folder):
    """Utility function returning an iterator over the csv files in given folder"""
    path = ''.join([DATA_FOLDER, '/ebola/', folder, '/'])
    for file in os.listdir(path):
        if file.endswith(".csv"):
            yield file, ''.join([path, file])

### Guinea

In [37]:
dfs = {}
tables_with_duplicates = []
for file, path in ebola_csv_iter('guinea_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Description', 'Totals'])
    df.rename(columns={'Description': 'Variable', 'Totals': 'National'}, inplace=True)
    df.Variable = df.Variable.astype('category')
    if len(df.Variable.unique()) != len(df.Variable): # Check for duplicates
        tables_with_duplicates.append(file)
    dfs[file] = df

In [38]:
tables_with_duplicates

[]

As there are no duplicates, we can pivot all the dataframes to have a timeseries dataframe:

In [39]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [40]:
dfs[0]

Variable,Cumulative (confirmed + probable + suspects),Fatality rate for confirmed and probables,New admits to CTE so far,New cases of confirmed,New cases of confirmed among health workers,New cases of probables,New cases of suspects,New contacts registered so far,New deaths registered,New deaths registered among health workers,...,Total deaths of confirmed,Total deaths of probables,Total deaths of suspects,Total deaths registered among health workers,Total new cases registered so far,Total number of admissions to CTE,Total number of exits from CTE,Total number of hospitalized cases in CTE,Total samples tested,Total suspected non-class cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-02,790,63%,29,14,0,0,11,0,5,0,...,349,150,2,26,25,1144,24,105,2018,642


In [8]:
guinea = pd.concat(dfs).sort_index() # Create the data series: index=date

In [9]:
guinea.head(n=10)

Variable,Cumulative (confirmed + probable + suspects),Fatality rate for confirmed and probables,New admits to CTE so far,New cases of confirmed,New cases of confirmed among health workers,New cases of probables,New cases of suspects,New contacts registered so far,New deaths registered,New deaths registered among health workers,...,Total new cases registered so far,Total number of admissions to CTE,Total number of exits from CTE,Total number of female cases,Total number of hospitalized cases in CTE,Total number of male cases,Total of cured in confirmed cases in CTE,Total of deaths in confirmed cases in CTE,Total samples tested,Total suspected non-class cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-08-04,495,NaN,NaN,4,NaN,0,5,NaN,NaN,NaN,...,9,NaN,NaN,277.0,NaN,220.0,89.0,138.0,NaN,NaN
2014-08-26,661,66%,22,10,0,NaN,18,137,5,0,...,28,997,6,NaN,87,NaN,NaN,NaN,1700,518
2014-08-27,663,66%,12,10,0,0,12,127,2,0,...,22,999,14,NaN,80,NaN,NaN,NaN,1719,518
2014-08-30,707,66%,9,9,0,0,15,21,5,0,...,24,1094,33,NaN,82,NaN,NaN,NaN,1858,560
2014-08-31,749,65%,26,29,0,8,9,0,3,0,...,46,1113,10,NaN,99,NaN,NaN,NaN,1941,642
2014-09-02,790,63%,29,14,0,0,11,0,5,0,...,25,1144,24,NaN,105,NaN,NaN,NaN,2018,642
2014-09-04,823,61%,13,17,0,0,13,21,5,0,...,30,1191,15,NaN,103,NaN,NaN,NaN,249,682
2014-09-07,861,65%,50,11,2,0,5,5,4,0,...,16,1256,8,NaN,102,NaN,NaN,NaN,2236,714
2014-09-08,861,65%,50,11,2,0,5,5,4,0,...,16,1256,8,NaN,102,NaN,NaN,NaN,2236,714


### Liberia

In [42]:
dfs = {}
tables_with_duplicates = []

for file, path in ebola_csv_iter('liberia_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Variable', 'National'])
    df.Variable = df.Variable.astype('category')
        
    if len(df.Variable.unique()) != len(df.Variable): # Check for duplicates
        tables_with_duplicates.append(file)
            
    dfs[file] = df

We remark that only a data file contains duplicate variables for the same day. Before dropping the duplicates, we look at the duplicated lines:

In [43]:
tables_with_duplicates

['2014-10-04-v142.csv']

In [44]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
8,2014-10-04,Cumulative (confirmed + probable + suspected),3921.0
10,2014-10-04,Cumulative cases among HCW,190.0
12,2014-10-04,Cumulative deaths among HCW,92.0
17,2014-10-04,Total death/s in confirmed cases,1012.0
18,2014-10-04,Total death/s in probable cases,699.0
19,2014-10-04,Total death/s in suspected cases,488.0
20,2014-10-04,"Total death/s in confirmed, probable, suspecte...",2199.0
31,2014-10-04,Cumulative (confirmed + probable + suspected),3929.0
32,2014-10-04,Cumulative cases among HCW,192.0
33,2014-10-04,Cumulative deaths among HCW,94.0


7 Variables are repeated twice.

In [45]:
first = with_duplicates[with_duplicates.duplicated(keep='last', subset='Variable')][['Variable', 'National']]
second = with_duplicates[with_duplicates.duplicated(keep='first', subset='Variable')][['Variable', 'National']]
first = first.set_index('Variable')
second = second.set_index('Variable')

In [46]:
second - first

,National
Variable,
Cumulative (confirmed + probable + suspected),8.0
Cumulative cases among HCW,2.0
Cumulative deaths among HCW,2.0
Total death/s in confirmed cases,6.0
Total death/s in probable cases,2.0
Total death/s in suspected cases,3.0
"Total death/s in confirmed, probable, suspected cases",11.0


We decided to keep the bigger values as the difference is not very relevant.

In [47]:
with_duplicates.drop_duplicates(subset='Variable', keep='last', inplace=True)

In [48]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [49]:
dfs[0]

Variable,Case Fatality Rate (CFR) - Confirmed & Probable Cases,Contacts lost to follow-up,Contacts seen,Contacts who completed 21 day follow-up,Cumulative admission/isolation,Cumulative cases among HCW,Cumulative deaths among HCW,Currently under follow-up,New Case/s (Probable),New Case/s (Suspected),...,Total contacts listed,Total death/s in confirmed cases,"Total death/s in confirmed, probable, suspected cases",Total death/s in probable cases,Total death/s in suspected cases,Total discharges,Total no. currently in Treatment Units,Total probable cases,Total specimens tested,Total suspected cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-10,NaN,3.0,3527.0,321.0,49.0,169.0,82.0,3583.0,28.0,29.0,...,7081.0,563.0,1296.0,472.0,261.0,2.0,49.0,1078.0,32.0,539.0


In [50]:
liberia = pd.concat(dfs).sort_index()

In [51]:
liberia.head(n=10)

Variable,Case Fatality Rate (CFR) - Confirmed & Probable Cases,Case Fatality Rate (CFR) - Confirmed & Probable Cases,Contacts lost to follow-up,Contacts seen,Contacts who completed 21 day follow-up,Contacts who completed 21 day follow-up,Cumulative (confirmed + probable + suspected),Cumulative (confirmed + probable + suspects),Cumulative CFR,Cumulative admission/isolation,...,"Total death/s in confirmed, probable, suspected cases","Total death/s in confirmed, probable, suspected cases",Total death/s in probable cases,Total death/s in suspected cases,Total discharges,Total no. currently in Treatment Units,Total no. currently in Treatment Units,Total probable cases,Total specimens tested,Total suspected cases
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-16,NaN,NaN,NaN,95.0,NaN,11.0,NaN,NaN,NaN,5.0,...,NaN,16.0,6.0,2.0,3.0,NaN,5.0,6.0,28.0,4.0
2014-06-17,NaN,NaN,0.0,90.0,NaN,11.0,NaN,NaN,NaN,10.0,...,NaN,16.0,6.0,2.0,0.0,NaN,10.0,6.0,34.0,8.0
2014-06-22,NaN,25.0,0.0,232.0,NaN,17.0,NaN,NaN,NaN,15.0,...,NaN,25.0,8.0,1.0,0.0,NaN,15.0,8.0,49.0,6.0
2014-06-24,NaN,NaN,15.0,224.0,NaN,17.0,NaN,NaN,NaN,25.0,...,NaN,32.0,8.0,6.0,0.0,NaN,25.0,8.0,63.0,13.0
2014-06-25,NaN,NaN,15.0,112.0,NaN,17.0,NaN,NaN,NaN,25.0,...,NaN,37.0,8.0,9.0,0.0,NaN,25.0,9.0,4.0,17.0
2014-06-28,NaN,61.0,15.0,407.0,NaN,22.0,NaN,NaN,NaN,25.0,...,NaN,49.0,10.0,13.0,3.0,NaN,25.0,18.0,3.0,29.0
2014-06-29,NaN,NaN,15.0,0.0,NaN,22.0,NaN,NaN,NaN,25.0,...,NaN,49.0,10.0,13.0,1.0,NaN,25.0,18.0,4.0,29.0
2014-07-01,NaN,70.1,15.0,422.0,NaN,17.0,NaN,NaN,NaN,19.0,...,NaN,61.0,17.0,14.0,3.0,NaN,19.0,19.0,4.0,33.0
2014-07-02,NaN,74.3,15.0,423.0,NaN,40.0,NaN,NaN,NaN,18.0,...,NaN,66.0,20.0,14.0,0.0,NaN,18.0,22.0,0.0,34.0


### Sierra Leone

In [52]:
dfs = {}
tables_with_duplicates = []

for file, path in ebola_csv_iter('sl_data'):
    df = pd.read_csv(path,
                     parse_dates=['date'],
                     usecols=['date', 'variable', 'National'])
    df.rename(columns={'date': 'Date', 'variable': 'Variable'}, inplace=True)
    df.Variable = df.Variable.astype('category')
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [53]:
tables_with_duplicates

['2014-11-10-v167.csv']

In [54]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
34,2014-11-10,pending,32.0
35,2014-11-10,positive_corpse,37.0
36,2014-11-10,negative_corpse,91.0
37,2014-11-10,pending,NaN
38,2014-11-10,positive_corpse,NaN
39,2014-11-10,negative_corpse,NaN


This time we decide to drop `NaN` values in favor of present data.

In [55]:
with_duplicates.drop_duplicates(subset='Variable', keep='first', inplace=True)

In [56]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [57]:
dfs[0]

Variable,cfr,contacts_followed,contacts_healthy,contacts_ill,contacts_not_seen,cum_completed_contacts,cum_confirmed,cum_contacts,cum_noncase,cum_probable,...,new_positive,new_probable,new_samples,new_suspected,pending,percent_seen,population,positive_corpse,repeat_samples,total_lab_samples
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-25,29.0,NaN,NaN,NaN,NaN,NaN,3560.0,NaN,3241.0,82.0,...,70.0,9.0,201.0,8.0,NaN,NaN,6348350.0,33.0,39.0,240.0


In [58]:
sl = pd.concat(dfs).sort_index()

In [59]:
sl.head(n=10)

Variable,cfr,contacts_followed,contacts_healthy,contacts_ill,contacts_not_seen,cum_completed_contacts,cum_confirmed,cum_contacts,cum_noncase,cum_probable,...,new_positive,new_probable,new_samples,new_suspected,pending,percent_seen,population,positive_corpse,repeat_samples,total_lab_samples
Date,,,,,,,,,,,,,,,,,,,,,
2014-08-12,36.8,2255,1937,14,131,901,717,2838,657,37,...,NaN,1,NaN,10,NaN,87%,6348350,NaN,NaN,NaN
2014-08-13,37.2,NaN,1956,14,131,NaN,733,2099,680,38,...,NaN,1,NaN,3,NaN,94%,"6,348,350",NaN,NaN,NaN
2014-08-14,37.5,2268,2135,7,117,1852,747,4173,708,39,...,NaN,2,NaN,0,NaN,94%,"6,348,350",NaN,NaN,NaN
2014-08-15,37.9,2212,2137,7,95,1927,757,4280,726,37,...,NaN,1,NaN,6,NaN,97%,"6,348,350",NaN,NaN,NaN
2014-08-16,38.3,2323,2265,6,79,2021,775,4361,741,34,...,NaN,0,NaN,3,NaN,98%,"6,348,350",NaN,NaN,NaN
2014-08-17,39.2,2428,2311,8,97,2135,778,4451,758,37,...,NaN,0,NaN,1,NaN,96%,"6,348,350",NaN,NaN,NaN
2014-08-18,39.8,2510,2420,16,66,2243,783,4581,758,52,...,NaN,15,NaN,40,NaN,97%,"6,348,350",NaN,NaN,NaN
2014-08-19,39.8,2434,2347,14,71,2313,804,4686,802,40,...,NaN,3,NaN,16,NaN,97%,6348350,NaN,NaN,NaN
2014-08-20,39.6,2423,2227,11,172,2384,813,4804,817,37,...,NaN,0,NaN,1,NaN,92%,6348350,NaN,NaN,NaN


In [60]:
del dfs
del tables_with_duplicates

### Combining the data

In [32]:
guinea.columns

CategoricalIndex(['Cumulative (confirmed + probable + suspects)',
                  'Fatality rate for confirmed and probables',
                  'New admits to CTE so far', 'New cases of confirmed',
                  'New cases of confirmed among health workers',
                  'New cases of probables', 'New cases of suspects',
                  'New contacts registered so far', 'New deaths registered',
                  'New deaths registered among health workers',
                  'New deaths registered today',
                  'New deaths registered today (confirmed)',
                  'New deaths registered today (probables)',
                  'New deaths registered today (suspects)',
                  'Number of confirmed cases among health workers',
                  'Number of contacts followed today',
                  'Number of contacts followed yesterday',
                  'Number of contacts lost to follow up',
                  'Number of contacts out of the trac

In [ ]:
liberia.columns

In [ ]:
sl.columns

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [24]:
# Setup for location of the dataset of the task 2
MICROBIOME_FOLDER = DATA_FOLDER + "/microbiome"

In [25]:
# Write your answer here


### Basic analysis of the file formatting:

***
**For the files MIDn.xls with n in [1,9]**

**column 1:**

We see that the first column in the files contain the scientific classification of the microbiomes
Altough it could be kept as a single string, it would have more meaning if splitted
The scientific classification contains the following subdivisions (https://en.wikipedia.org/wiki/Taxonomic_rank): 

       Domain, Kingdom, Phylum, Class, Order, Family, Genus, Species
       
 With this classification, we have a problem: only 6 strings are given in the data set, whilst we have 8 potential divisions in the classification.
 
 After analysis, it seems that Kingdom is not used with bacterias (https://en.wikipedia.org/wiki/Bacteria)
 and always the same as the Phylum in Archea(https://en.wikipedia.org/wiki/Archaea). Therefore, **we will not use Kingdom as a division**.
 
 Also, Species only has a sense in the Eucaryote Domain, for which we have no data, so **we will not use Species as a division**.

In [26]:
# Lets keep those names in a list for further use
scientific_classification = ["Domain", "Phylum", "Class", "Order", "Family", "Genus"]

**Column 2:**

We see that the second column contains an integer value. We can suppose that this value is the number of samples
containing the genus defined by the first column. There's no title to the column, so we don't know yet where
those values came from.

***

**For the file "Metadata.xls"**

**Column 1:**

Titled **"BARCODE"**, give xls file identifier for which the two other columns give more information

**Column 2:**

Titled **"GROUP"**, gives the groups from which each dataset has been sampled. Two informations are contained in those groups, the first is given by "NEC", "Control" or "EXTRACTION CONTROL" and the second is the numbering of the group (which is likely a phase of test), either "1" or "2". We will split those informations in two column, because while using the dataset, we might want to combine all the "NEC" patients or all the patient for a specific phase.

**Column 3:**

Titled **"SAMPLE"**, gives the type of sample that was taken, either tissue, stool or NA. Each group had both types of samples taken. 

***

### Desired formating of the data after analysis

The simple analysis above tells us what are the columns that we will want in our Data Frame

1. **6** column for classification, 1 for each classifier in the "scientific_classification" list. This will caracterise each microbiome individually.

2. **1** column will contain the value associated with each microbiome measurement.

3. **2** columns describing the group of the sample from the metadata. The first column will be called **Group Type** and will contain the "NEC", "Control" or "EXTRACTION CONTROL" value. The second column will be called **Group Phase** and will contain either "1", "2" or "unknow", "unknow" will be used for the "EXCTRACTON CONTROL" group.

4. **1** column describing the type of sample taken from the metadata. This column will be called **Sample** This will be either "tissue", "stool" or "unknow", "unknow" in the case of the EXTRACTION CONTROL group.

For a total of **10** columns


In [61]:
df_col_list = scientific_classification + ["Value", "Group Type", "Group Phase", "Sample"]
df_col_list

['Domain',
 'Phylum',
 'Class',
 'Order',
 'Family',
 'Genus',
 'Value',
 'Group Type',
 'Group Phase',
 'Sample']

***
### We want to extract the metadata from the metadata.xls file

The metadata is needed before we start extracting the data from the MB files to create the DataFrame with all the desired columns.


In [98]:
#The actual name of the excel sheet is "Sheet1" and not "Sheet 1" as in the other files.
metadata_raw = pd.read_excel(MICROBIOME_FOLDER+"/metadata.xls", sheetname='Sheet1', header=0)
metadata_raw


,BARCODE,GROUP,SAMPLE
0,MID1,EXTRACTION CONTROL,NaN
1,MID2,NEC 1,tissue
2,MID3,Control 1,tissue
3,MID4,NEC 2,tissue
4,MID5,Control 2,tissue
5,MID6,NEC 1,stool
6,MID7,Control 1,stool
7,MID8,NEC 2,stool
8,MID9,Control 2,stool


In [111]:
## Extract group phase and group type
group = metadata_raw.GROUP
group_type = []
group_phase = []
for index in range(len(group)):                # Index progression must be upward!
    # A special case for the extraction control, we don't want to split it
    if group[index] == "EXTRACTION CONTROL":
        group_type.append(group[index])
        group_phase.append("unknow")
    else:
        splitted_group = group[index].split()
        group_type.append(splitted_group[0])
        group_phase.append(splitted_group[1])
        

In [112]:
metadata_raw['group_type'] = pd.Series(group_type)
metadata_raw['group_phase'] = pd.Series(group_phase)
metadata = metadata_raw.drop("GROUP",  axis=1)
metadata

,BARCODE,SAMPLE,group_type,group_phase
0,MID1,NaN,EXTRACTION CONTROL,unknow
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


In [113]:
## make sure SAMPLE doesn't contain NaN
metadata.SAMPLE.fillna(value="unknow", inplace=True)
metadata

,BARCODE,SAMPLE,group_type,group_phase
0,MID1,unknow,EXTRACTION CONTROL,unknow
1,MID2,tissue,NEC,1
2,MID3,tissue,Control,1
3,MID4,tissue,NEC,2
4,MID5,tissue,Control,2
5,MID6,stool,NEC,1
6,MID7,stool,Control,1
7,MID8,stool,NEC,2
8,MID9,stool,Control,2


***
### We now want to extract the data from the datasheets

** We create a function to extract the classifications **

We want to extract the classifications from the first column of the datasheets. We will need to do string parsing. Regex are a great tool to remove the unwanted characters from the format. The only unwanted characters are the double quotes ".

In [39]:
import re

# Next function replace " in the classifier_string, split the string to have an indexable list and return it
def get_classifiers(classifier_string):
    return re.subn("\"", "", classifier_string)[0].split()

In [52]:
mb1 = pd.read_excel(MICROBIOME_FOLDER+"/MID1.xls", sheetname='Sheet 1', header=None)


get_classifiers(mb1[0][0])

#classified_mb1 = pd.DataFrame([classifiers], columns=scientific_classification,
 #                             "Value", "Group Type", "Group Phase", "Sample")

#classified_mb1
#for index in range(len(mb1[0])):
    



['Archaea',
 'Crenarchaeota',
 'Thermoprotei',
 'Desulfurococcales',
 'Desulfurococcaceae',
 'Ignisphaera']

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here